# Reflexion

In [61]:
from dotenv import load_dotenv

load_dotenv()

True

In [62]:
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel

In [63]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [71]:
class AgentState(BaseModel):
    task: str = ""
    code: str = ""
    error: str = ""
    history: list = []
    result: str = ""
    reflection: str = ""
    reflection_count: int = 0

In [65]:
TASK_PROMPT = ChatPromptTemplate.from_messages([
    ("system", "You're an expert Python developer. Write Python code to: {task}. Please give me the code only. No extra details are needed."),
    ("user", "Previous Error (if any): {error}\n\nPlease write code again."),
])

REFLECT_PROMPT = ChatPromptTemplate.from_messages([
    ("system", "You are debugging your own code. Reflect on the following error and suggest how to fix it."),
    ("user", "Code: {code}\n\nError: {error}\n\nWhat went wrong?\n\nHistory: {history}\n\n"),
])

In [77]:
import traceback

def generate_code(state: AgentState) -> AgentState:
    task = state.task
    error = state.error
    messages = TASK_PROMPT.format_messages(task=task, error=error)
    output = llm.invoke(messages)
    state.code = output.content
    state.history.append(output.content)
    return state


def execute_code(state: AgentState) -> AgentState:
    code = state.code
    try:
        exec_globals = {}
        exec(code, exec_globals)
        state.result = "success"
    except Exception as e:
        state.result = "fail"
        state.error = traceback.format_exc()

    return state
    

def reflect(state: AgentState) -> AgentState:
    state.reflection_count += 1
    code = state.code
    error = state.error
    messages = REFLECT_PROMPT.format_messages(code=code, error=error, history=state.history)
    reflection = llm.invoke(messages)
    state.reflection = reflection.content
    return state

In [81]:
def should_reflect(state: AgentState):
    print(f"Reflection count: {state.reflection_count}")
    if state.reflection_count >= 3:
        return END
    return "reflect" if state.result == "fail" else END

In [82]:
workflow = StateGraph(AgentState)
workflow.add_node("generate_code", RunnableLambda(generate_code))
workflow.add_node("execute_code", RunnableLambda(execute_code))
workflow.add_node("reflect", RunnableLambda(reflect))

workflow.set_entry_point("generate_code")
workflow.add_edge("generate_code", "execute_code")
workflow.add_conditional_edges("execute_code", should_reflect)
workflow.add_edge("reflect", "generate_code")

In [83]:
app = workflow.compile()

initial_state = AgentState(task="Write a function that divides two numbers, but handles divide-by-zero errors safely.")
final_state = AgentState(**app.invoke(initial_state, {"recursion_limit": 100}))

Reflection count: 0
Reflection count: 1
Reflection count: 2
Reflection count: 3


In [84]:
print("Final Result:\n")
print(final_state.code)
print("\nReflection History:")

for item in final_state.history:
    print("---\n", item)
    
if final_state.result == "fail":
    print("\nStill failed after reflexion attempts")

Final Result:

```python
def safe_divide(a, b):
    try:
        result = a / b
    except ZeroDivisionError:
        result = None
    return result
```

Reflection History:
---
 Sure, here is the code:

```python
def safe_divide(a, b):
    try:
        result = a / b
    except ZeroDivisionError:
        print("Error: Division by zero is not allowed")
        result = None
    return result
```
---
 ```python
def safe_divide(num, denom):
    try:
        result = num / denom
    except ZeroDivisionError:
        result = "Error: Cannot divide by zero"
    return result
```
---
 ```python
def safe_divide(a, b):
    try:
        result = a / b
    except ZeroDivisionError:
        print("Error: Division by zero")
        result = None
    return result
```
---
 ```python
def safe_divide(a, b):
    try:
        result = a / b
    except ZeroDivisionError:
        result = None
    return result
```

Still failed after reflexion attempts
